## Tools for building Henderson's Mixed Model Equations

Use of Gibbs sampler to compute posterior mean of effects 


In [30]:
include("../../PedModule.jl/src/PedModule.jl")
using DataFrames
using Distributions

In [31]:
df = readtable("MTData", separator = ' ')

,y1,y2,y3,trt
1,1.0,2.0,1.2,1
2,1.1,NA,3.1,1
3,0.9,1.9,NA,2
4,1.2,1.7,1.9,2


In [32]:
function Gibbs(A,x,b,nIter;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVarInv = 1.0/A[i,i]
            cMean   = cVarInv*(b[i] - A[:,i]'x)[1,1] + x[i]
            x[i]    = randn()*sqrt(cVarInv) + cMean 
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

function Gibbs(A,x,b)
    n = size(x,1)
    for i=1:n
        cVarInv = 1.0/A[i,i]
        cMean   = cVarInv*(b[i] - A[:,i]'x)[1,1] + x[i]
        x[i]    = randn()*sqrt(cVarInv) + cMean 
    end
end

function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        #println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end


type TermStrVal
    str::AbstractString
    value::Float64
end

type TermLvlVal
    level::AbstractString
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}            # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end
type MME
    modelVec::Array{AbstractString,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{AbstractString,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    X
    ySparse
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    R::Array{Float64,2}
    Ai
    mmePos::Int64
end

type ResVar
    R0::Array{Float64,2}
    RiDict::Dict{BitArray{1},Array{Float64,2}}
end   

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getRi(resVar::ResVar,sel::BitArray{1})
    if haskey(resVar.RiDict,sel)
        return resVar.RiDict[sel]
    end
    n = size(resVar.R0,1)
    RZ = zeros(n,n)
    RZ[sel,sel] = inv(resVar.R0[sel,sel])
    resVar.RiDict[sel] = copy(RZ)
    return RZ
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec;symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms]
        for (i,trm) = enumerate(modelTerms) 
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,0,0,Array(Float64,1,1),R,0,1)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str]) 
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function mkRi(mme::MME,df::DataFrame)
    resVar = ResVar(mme.R,Dict())
    tstMsng = !isna(df[mme.lhsVec[1]])
    for i=2:size(mme.lhsVec,1)
        tstMsng = [tstMsng !isna(df[mme.lhsVec[i]])]
    end
    n    = size(tstMsng,2)
    nObs = size(tstMsng,1)
    ii = Array(Int64,nObs*n^2)
    jj = Array(Int64,nObs*n^2)
    vv = Array(Float64,nObs*n^2)
    pos = 1
    for i=1:size(tstMsng,1)
        sel = reshape(tstMsng[i,:],n)
        Ri  = getRi(resVar,sel)
        for ti=1:n
            tii = (ti-1)*nObs + i
            for tj=1:n
                tjj = (tj-1)*nObs + i
                ii[pos] = tii
                jj[pos] = tjj
                vv[pos] = Ri[ti,tj]
                pos += 1
            end
        end         
    end
    return sparse(ii,jj,vv)
end

function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = convert(Array,df[mme.lhsVec[1]],0.0)
    for i=2:size(mme.lhsVec,1)
        y    = [y; convert(Array,df[mme.lhsVec[i]],0.0)] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = mkRi(mme,df)
    mme.X = X
    mme.ySparse = ySparse 
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme)
    end   
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ",keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res;string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

addA (generic function with 1 method)

In [33]:
ped = PedModule.mkPed("sim.ped");

In [34]:
dfGen = readtable("sim.gen", separator = ' ');

In [35]:
Q = convert(Array{Float64,2},dfGen[:,collect(2:end)]);
α1 = randn(200)
α2 = randn(200)
a1 = Q*α1
a2 = Q*α2;

In [36]:
D = diagm(vec(sqrt(var([a1 a2],1))'))

2x2 Array{Float64,2}:
 9.32736  0.0    
 0.0      7.53884

In [37]:
R = cor([a1 a2])

2x2 Array{Float64,2}:
  1.0        -0.0923781
 -0.0923781   1.0      

In [38]:
G0 = D*R*D

2x2 Array{Float64,2}:
 86.9996   -6.49579
 -6.49579  56.834  

In [39]:
R0 = diagm(vec(var([a1 a2],1)))
L  = chol(R0)
e  = L*randn(2,size(Q,1))
y = [a1 a2] + e'

4099x2 Array{Float64,2}:
 -21.776      46.4939 
   2.27156    33.0493 
   2.53801    14.2841 
  -5.93039    17.4044 
   9.86479   -15.5682 
  -6.83143    11.5414 
 -12.3529      2.87854
  -7.12516    28.7472 
  -5.21745    15.8114 
 -16.5868     34.0291 
 -12.3176     15.3733 
  -4.82797    18.8197 
 -35.3802     24.2964 
   ⋮                  
  -6.02472    20.0302 
 -18.8907     18.522  
   0.058964   24.3469 
 -17.6329     44.2426 
  -1.76942    32.3374 
  21.5065     25.0077 
   0.689033   27.0574 
  -0.511425   12.09   
   4.24939    15.9652 
 -12.7776     26.5019 
 -11.2318     30.4111 
   3.78635     8.58891

In [40]:
df2 = DataFrame(Animal = dfGen[:,1], y1=y[:,1],y2=y[:,2])

,Animal,y1,y2
1,10102,-21.775987397669763,46.493850748967446
2,10103,2.271560757462078,33.04925811459783
3,10104,2.5380064351683043,14.284068969252191
4,10105,-5.930393402911851,17.40444749887416
5,10106,9.864791824975551,-15.568195170912201
6,10107,-6.831429269336013,11.541363724575806
7,10108,-12.352919250638461,2.878536903726012
8,10109,-7.125155713912806,28.74719079534854
9,10110,-5.2174466766319405,15.811384577034854
10,10111,-16.586767852926567,34.029109519417524


In [41]:
models = "y1 = intercept + Animal;
          y2 = intercept + Animal"
R = R0
mme = initMME(models,R)
setAsRandom(mme,"Animal", ped,G0)

In [52]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    getMME(mme,df2)
    p = size(mme.mmeLhs,1)
    sol = fill(0.0,p)
    solMean = fill(0.0,p)
    GaussSeidel(mme.mmeLhs,sol,mme.mmeRhs,tol=0.000001) 
    ν = 10
    nObs    = size(df,1)
    nTraits = size(mme.lhsVec,1)
    νR0 = ν + nTraits
    R0 = mme.R
    PRes = R0*(νR0 - nTraits - 1)
    SRes   = zeros(Float64,nTraits,nTraits)
    R0Mean = zeros(Float64,nTraits,nTraits)
    if mme.ped != 0
        pedTrmVec = mme.pedTrmVec
        k = size(pedTrmVec,1)
        νG0 = ν + k
        G0 = inv(mme.Gi)
        P = G0*(νG0 - k - 1)
        S = zeros(Float64,k,k)
        G0Mean = zeros(Float64,k,k)
    end
    for iter=1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
            println(G0Mean)
        end
        Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
        # can make this more efficient by taking advantage of symmetry
        for (i,trmi) = enumerate(pedTrmVec)    
            pedTrmi  = mme.modelTermDict[trmi]
            startPosi  = pedTrmi.startPos
            endPosi    = startPosi + pedTrmi.nLevels - 1
            for (j,trmj) = enumerate(pedTrmVec)
                pedTrmj  = mme.modelTermDict[trmj]
                startPosj  = pedTrmj.startPos
                endPosj    = startPosj + pedTrmj.nLevels - 1
                S[i,j] = (sol[startPosi:endPosi]'*mme.Ai*sol[startPosj:endPosj])[1,1]
            end
        end
        resVec = mme.ySparse - mme.X*sol
        for traiti = 1:nTraits
            startPosi = (traiti-1)*nObs + 1
            endPosi   = startPosi + nObs - 1
            for traitj = traiti:nTraits
                startPosj = (traitj-1)*nObs + 1
                endPosj   = startPosj + nObs - 1
                SRes[traiti,traitj] = (resVec[startPosi:endPosi]'resVec[startPosj:endPosj])[1,1] 
                SRes[traiti,traitj] = SRes[traitj,traiti]
            end
        end
        R0 = rand(InverseWishart(νR0 + nObs, PRes + SRes))
        mme.R = R0
        Ri = mkRi(mme,df)
        X = mme.X
        mme.mmeLhs = X'Ri*X
        mme.mmeRhs = X'Ri*mme.ySparse
        if mme.ped != 0
            pedTrm1 = mme.modelTermDict[pedTrmVec[1]]
            q = pedTrm1.nLevels
            G0 = rand(InverseWishart(νG0 + q, P + S))
            mme.Gi = inv(G0)
            addA(mme)
        end
        solMean += (sol - solMean)/iter
        G0Mean  += (G0  - G0Mean )/iter
        R0Mean  += (R0  - R0Mean )/iter
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = solMean
    output["posteriorMeanG0"] = G0Mean
    output["posteriorMeanR0"] = R0Mean
    return output
end

sampleMCMC (generic function with 1 method)

In [53]:
@time res = sampleMCMC(2000,mme,df2)
res["posteriorMeanG0"]

at sample: 100
[109.28025515104126 -8.969725878464528
 -8.969725878464528 55.62415605152393]
at sample: 200
[115.1500176122906 -8.907312862903781
 -8.907312862903781 58.10265043818913]
at sample: 300
[114.34633913997473 -8.7876540887398
 -8.7876540887398 56.82202182741817]
at sample: 400
[114.63464601197298 -8.212604120260073
 -8.212604120260073 53.82122386787074]
at sample: 500
[113.99429720466325 -8.312183723330374
 -8.312183723330374 55.920357909232514]
at sample: 600
[116.53009338957753 -8.083862944729976
 -8.083862944729976 55.32175969299899]
at sample: 700
[116.22985303466982 -8.245253506356885
 -8.245253506356885 55.37739458834972]
at sample: 800
[116.32164645561949 -8.12328260432483
 -8.12328260432483 56.13459715394661]
at sample: 900
[115.6599047035378 -8.134048737486024
 -8.134048737486024 56.287330277809296]
at sample: 1000
[115.035369387408 -8.36889564700102
 -8.36889564700102 55.33576347258628]
at sample: 1100
[115.88541556116628 -8.466909353430468
 -8.466909353430468 53.7

2x2 Array{Float64,2}:
 117.154    -7.99173
  -7.99173  52.2808 


[117.1490946493896 -7.991299630316052
 -7.991299630316052 52.28665535974951]
139.540956 seconds (782.81 M allocations: 70.727 GB, 6.61% gc time)


In [44]:
res["posteriorMeanR0"]

2x2 Array{Float64,2}:
 96.9679     -0.0615604
 -0.0615604  77.2945   


141.269832 seconds (782.72 M allocations: 70.719 GB, 6.84% gc time)


In [46]:
var([a1 a2],1)

1x2 Array{Float64,2}:
 86.9996  56.834

In [48]:
var(e',1)

1x2 Array{Float64,2}:
 90.5453  58.281

In [49]:
var([a1 a2],1) + var(e',1)

1x2 Array{Float64,2}:
 177.545  115.115

In [51]:
var(y,1)

1x2 Array{Float64,2}:
 180.297  116.304